In [1]:
import h2o
from h2o.estimators.word2vec import H2OWord2vecEstimator
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint
from datetime import datetime
import random
from feature_engine.imputation import MeanMedianImputer
from sklearn.cluster import KMeans

In [2]:
okhttp_commits_df = pd.read_csv('raw_commits_github_json/Camel/Camel_commits.csv')

In [3]:
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring
0,fe14d164ac23f99c3c5da815a56c1fae7fd6a178,Otavio Rodolfo Piske,opiske@redhat.com,2021-11-09 10:06:31,orpiske,CAMEL-17121: converted camel-splunk to repeata...,62147da2c25b72019375acfc469a4413ba18f1c9,2021-11-09 10:05:37,0.0,0
1,62147da2c25b72019375acfc469a4413ba18f1c9,aldettinger,aldettinger@gmail.com,2021-11-09 10:05:37,aldettinger,CAMEL-11178: Fix wrong HTTP link to Woodstox,79aab5dcf7154477886b4abbf3e9212b58c91db3,2021-11-09 09:56:37,0.0,0
2,79aab5dcf7154477886b4abbf3e9212b58c91db3,aldettinger,aldettinger@gmail.com,2021-11-09 09:56:37,aldettinger,CAMEL-17178: Fix wrong HTTP links to Jackson J...,6998077b5bd7090617d2fbee45c55c5f8d40848c,2021-11-09 09:34:36,0.0,0
3,6998077b5bd7090617d2fbee45c55c5f8d40848c,aldettinger,aldettinger@gmail.com,2021-11-09 09:34:36,aldettinger,CAMEL-17178: Fix wrong HTTP links to Jackson l...,7b9a10c7bebc34ee6bde40b667156dd7f4612464,2021-11-09 09:21:23,0.0,0
4,7b9a10c7bebc34ee6bde40b667156dd7f4612464,aldettinger,aldettinger@gmail.com,2021-11-09 09:21:23,aldettinger,CAMEL-17178: Remove the useless reference to w...,30494c20e104ffdd2d347966f836d079f5f97d36,2021-11-09 08:32:59,0.0,0


In [4]:
len(okhttp_commits_df)

53329

In [5]:
okhttp_commits_df = okhttp_commits_df[okhttp_commits_df['time_taken'] > 5]
okhttp_commits_df = okhttp_commits_df[okhttp_commits_df['time_taken'] < 168]
len(okhttp_commits_df)

6798

In [6]:
okhttp_commits_df['message'].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.12+8-LTS-237, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\tanji\AppData\Local\Temp\tmpyju1706j
  JVM stdout: C:\Users\tanji\AppData\Local\Temp\tmpyju1706j\h2o_tanji_started_from_python.out
  JVM stderr: C:\Users\tanji\AppData\Local\Temp\tmpyju1706j\h2o_tanji_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Singapore
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.7
H2O_cluster_version_age:,2 months and 12 days
H2O_cluster_name:,H2O_from_python_tanji_3v482q
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.984 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"accepting new members, healthy"


In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [9]:
STOP_WORDS = stopwords.words('english')
STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can","lines","re","what",
               "there","all","we","one","the","a","an","of","or","in","for","by","on",
               "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
               "from","at","my","be","by","not","that","to","from","com","org","like","likes","so"]
STOP_WORDS = ['']



In [10]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [11]:
def create_token_table(data_hdf, col_name, prefix, sent_sample_rate = 0.0, epochs = 25, vec_size=20):
    # Tokenize
    words = tokenize(data_hdf[col_name])
    w2v_model = H2OWord2vecEstimator(sent_sample_rate = sent_sample_rate, 
                                     epochs = epochs, 
                                     vec_size = vec_size)
    w2v_model.train(training_frame=words)
    
    # Create the training vectors
    vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")
    
    
    # Prefix naming the columns
    vecs.columns = ['%s%s' % (prefix, c) for c in vecs.columns]
    data2_hdf = data_hdf.cbind(vecs)
    return data2_hdf, w2v_model

    
#     data3_hdf = data2_hdf[mask,:].cbind(vecs[mask,:])

In [12]:
commit_message_df = okhttp_commits_df[['message',]]

In [13]:
commit_message_df.shape

(6798, 1)

In [14]:
commit_message_df.head()

,message
5,CAMEL-17178: Fix wrong jackson 2 HTTP links
7,CAMEL-17121: converted camel-soroush to use th...
9,CAMEL-17121: converted camel-smpp to the repea...
12,CAMEL-17121: fixed time+iteration bounded supp...
22,Upgrade AWS SDK v2 to version 2.17.77


In [15]:
commit_message_hdf = h2o.H2OFrame(commit_message_df, 
                         column_types = {'message': 'string'}
                        )
commit_message_hdf.types

Parse progress: |█████████████████████████████████████████████████████████| 100%


{'message': 'string'}

In [16]:
commit_message_hdf.head()

message
CAMEL-17178: Fix wrong jackson 2 HTTP links
CAMEL-17121: converted camel-soroush to use the repeatable tasks
CAMEL-17121: converted camel-smpp to the repeatable tasks
CAMEL-17121: fixed time+iteration bounded support for background tasks
Upgrade AWS SDK v2 to version 2.17.77
Upgrade AWS SDK v2 to version 2.17.76
camel-as2-api sometimes fail compile on CI so modifying pom.xml to try to fix this.
CAMEL-17172: Allow to comfigure muteException on component level.
"StaxConverter: Make input and output factory pool size implementation (#6380) * StaxConverter: Make input and output factory pool size implementation match comments. Comments suggests that Runtime#availableProcessors should be used as pool size when it is greater than 20, but the current implementation uses Runtime#availableProcessors regardless of the amount of processors available if the system property ""org.apache.cxf.staxutils.pool-size"" is not set. * StaxConverter: Stop using the system property ""org.apache.cxf.staxutils.pool-size"" when determining the input and output factory pool size."
CAMEL-17162: Added unit test. Thanks to Jeremy Ross for the test.


In [17]:
# Train owner names model
commit_message_vecs_hdf, commit_message_w2v_model = create_token_table(commit_message_hdf,
                                                                  'message',
                                                                   prefix='message_',
                                                                   vec_size=30)

'''
words = tokenize(commit_message_hdf['message'])
w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 20, 
                                 vec_size = 30)
w2v_model.train(training_frame=words)

# Create the training vectors
vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")


# Prefix naming the columns
vecs.columns = ['%s%s' % ('message_', c) for c in vecs.columns]
data2_hdf = commit_message_hdf.cbind(vecs)
print(data2_hdf)

wv_path = 'models/'
model_path = h2o.save_model(model = w2v_model, path= wv_path ,force=True)

#Load Model
w2v_model2 = h2o.load_model(model_path)
'''

word2vec Model Build progress: |██████████████████████████████████████████| 100%


'\nwords = tokenize(commit_message_hdf[\'message\'])\nw2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 20, \n                                 vec_size = 30)\nw2v_model.train(training_frame=words)\n\n# Create the training vectors\nvecs = w2v_model.transform(words, aggregate_method = "AVERAGE")\n\n\n# Prefix naming the columns\nvecs.columns = [\'%s%s\' % (\'message_\', c) for c in vecs.columns]\ndata2_hdf = commit_message_hdf.cbind(vecs)\nprint(data2_hdf)\n\nwv_path = \'models/\'\nmodel_path = h2o.save_model(model = w2v_model, path= wv_path ,force=True)\n\n#Load Model\nw2v_model2 = h2o.load_model(model_path)\n'

In [18]:
commit_message_w2v_model.find_synonyms('refactor')

OrderedDict([('config', 0.6630274057388306),
             ('change', 0.651976466178894),
             ('datasonnet', 0.6376091241836548),
             ('where', 0.614065945148468),
             ('actual', 0.6099859476089478),
             ('makes', 0.6077536940574646),
             ('getter', 0.6022452116012573),
             ('query', 0.6008857488632202),
             ('project', 0.5900629162788391),
             ('method', 0.5829784274101257),
             ('base', 0.5827769041061401),
             ('made', 0.5821027755737305),
             ('annotation', 0.5809985399246216),
             ('reuse', 0.5770891904830933),
             ('bump', 0.574775755405426),
             ('compatibility', 0.5690039992332458),
             ('proxy', 0.5631380677223206),
             ('junit', 0.5543242692947388),
             ('revert', 0.5529279708862305),
             ('moving', 0.5506044030189514)])

In [19]:
rand_list = sorted( random.sample(range(commit_message_vecs_hdf.nrow), 4 ) )
commit_message_vecs_hdf[rand_list, :]

message,message_C1,message_C2,message_C3,message_C4,message_C5,message_C6,message_C7,message_C8,message_C9,message_C10,message_C11,message_C12,message_C13,message_C14,message_C15,message_C16,message_C17,message_C18,message_C19,message_C20,message_C21,message_C22,message_C23,message_C24,message_C25,message_C26,message_C27,message_C28,message_C29,message_C30
Upgrade Async Http client to version 4.1.3,0.352739,-0.173542,0.118257,-0.575162,0.369554,-0.291588,-0.3187,0.0571374,0.181543,0.241998,-0.226281,0.659644,0.0313362,0.415657,-0.188175,-0.0878539,0.458217,-0.216973,0.0205447,-0.422854,0.0144951,0.184376,-0.356818,0.0255562,0.726419,0.0939135,-0.734229,-0.115532,0.279941,-0.439511
CAMEL-10244: Fix code to compile,-0.145999,0.252559,-0.296519,-0.058314,-0.00214621,-0.0683982,-0.125718,0.210038,0.236668,-0.27756,0.158212,-0.0605965,0.0738177,0.57266,-0.31116,0.133944,0.216833,-0.0149309,0.117821,0.146801,0.202634,-0.208869,-0.161863,0.335614,0.196089,0.216324,-0.158351,-0.203156,-0.0810038,0.142551
CAMEL-6621 Add ability to connect to SQS queue of different owner with thanks to Jeff,0.396189,0.0713366,-0.0651564,-0.0348977,0.367486,0.12111,-0.382226,0.00491713,0.0976024,-0.202443,0.0114568,0.368584,0.00879731,0.400081,0.0155626,0.207832,0.116075,-0.223422,0.173305,0.271917,-0.0526369,-0.103826,-0.110754,0.175614,0.110205,0.048474,-0.145164,-0.183208,0.150788,-0.197828
Make the normalizer example look a bit nicer git-svn-id: https://svn.apache.org/repos/asf/camel/trunk@755341 13f79535-47bb-0310-9956-ffa450edef68,-0.339017,0.539766,-0.126502,-0.11891,0.387252,-0.0621721,-0.308567,0.0979191,0.52363,-0.303772,0.298004,-0.0324144,0.242159,0.0619474,-0.0660672,0.627409,0.508021,-0.174109,0.297097,-0.0680296,-0.256347,-0.0920663,-0.040411,0.363537,0.195146,0.438158,0.304884,-0.411785,0.223842,-0.344562


In [20]:
kmean = KMeans(n_clusters=10)

In [21]:
commit_message_vecs_hdf = commit_message_vecs_hdf.as_data_frame()

In [22]:
commit_message_vecs_hdf = commit_message_vecs_hdf.fillna(0)

In [23]:
commit_message_vecs_hdf.iloc[:,1:]

,message_C1,message_C2,message_C3,message_C4,message_C5,message_C6,message_C7,message_C8,message_C9,message_C10,...,message_C21,message_C22,message_C23,message_C24,message_C25,message_C26,message_C27,message_C28,message_C29,message_C30
0,-0.046703,0.101219,-0.146875,-0.203431,-0.049483,0.036745,-0.383202,0.240640,0.184352,-0.233439,...,0.168662,-0.298344,-0.273655,0.413035,0.143343,0.273259,-0.288878,-0.167644,0.187130,0.149241
1,0.451227,0.390624,-0.240798,-0.241086,0.058304,0.057937,-0.143853,0.312222,0.130050,-0.199219,...,0.150432,-0.108201,0.244740,0.326380,0.122385,-0.047701,-0.027357,-0.204425,0.238779,-0.448167
2,0.488023,0.402187,-0.224764,-0.200877,0.133683,0.075344,-0.096267,0.331890,0.113092,-0.247015,...,0.166164,-0.081767,0.316348,0.366035,0.088124,0.094085,-0.004722,-0.207634,0.220456,-0.430544
3,0.381105,0.352318,-0.231028,-0.168334,0.031963,0.075438,-0.125653,0.170660,0.199325,-0.377862,...,0.300696,-0.092064,0.059061,0.257540,0.264860,-0.245315,-0.186234,-0.240449,0.416335,-0.223639
4,-0.125230,-0.219510,0.187311,-1.210782,0.803214,-0.173714,0.523547,0.505062,0.190299,-0.093326,...,-0.346238,-0.313440,-0.118772,0.167825,1.265547,-0.016372,-0.814946,-0.053619,-0.114785,-0.680513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6793,-0.015958,0.171315,-0.395193,-0.020495,-0.001946,-0.092371,-0.308229,0.049486,-0.090308,-0.222593,...,0.097754,0.025827,0.186487,0.564189,0.116247,0.291977,-0.038142,-0.042308,0.183717,-0.073110
6794,0.114990,0.256889,-0.229672,-0.022887,-0.019319,0.156599,-0.483909,-0.007472,-0.093288,-0.452425,...,0.294520,0.126803,0.033299,0.498673,0.091987,0.228599,0.001935,-0.182807,0.031928,-0.121227
6795,-0.481139,-0.107158,-0.663608,-0.063583,-0.301917,-0.179333,-0.468309,0.003603,-0.127527,-0.826095,...,-0.069642,-0.611664,0.551355,0.744642,0.050071,-0.080886,-0.939807,-0.556583,-0.987845,0.336796
6796,0.460373,0.067875,-0.555457,0.120034,0.157488,0.406941,-0.278688,-0.159113,0.171704,-0.385773,...,0.064618,-0.167763,-0.206491,0.117048,0.000314,-0.218216,-0.101422,-0.499606,0.074264,-0.310593


In [24]:
kmean.fit(commit_message_vecs_hdf.iloc[:,1:])

KMeans(n_clusters=10)

In [25]:
kmean.labels_

array([8, 3, 3, ..., 5, 7, 7])

In [26]:
print(len(kmean.labels_))

6798


In [27]:
commit_message_vecs_hdf['kmean_label'] = kmean.labels_

In [28]:
commit_message_vecs_hdf.head()

,message,message_C1,message_C2,message_C3,message_C4,message_C5,message_C6,message_C7,message_C8,message_C9,...,message_C22,message_C23,message_C24,message_C25,message_C26,message_C27,message_C28,message_C29,message_C30,kmean_label
0,CAMEL-17178: Fix wrong jackson 2 HTTP links,-0.046703,0.101219,-0.146875,-0.203431,-0.049483,0.036745,-0.383202,0.240640,0.184352,...,-0.298344,-0.273655,0.413035,0.143343,0.273259,-0.288878,-0.167644,0.187130,0.149241,8
1,CAMEL-17121: converted camel-soroush to use th...,0.451227,0.390624,-0.240798,-0.241086,0.058304,0.057937,-0.143853,0.312222,0.130050,...,-0.108201,0.244740,0.326380,0.122385,-0.047701,-0.027357,-0.204425,0.238779,-0.448167,3
2,CAMEL-17121: converted camel-smpp to the repea...,0.488023,0.402187,-0.224764,-0.200877,0.133683,0.075344,-0.096267,0.331890,0.113092,...,-0.081767,0.316348,0.366035,0.088124,0.094085,-0.004722,-0.207634,0.220456,-0.430544,3
3,CAMEL-17121: fixed time+iteration bounded supp...,0.381105,0.352318,-0.231028,-0.168334,0.031963,0.075438,-0.125653,0.170660,0.199325,...,-0.092064,0.059061,0.257540,0.264860,-0.245315,-0.186234,-0.240449,0.416335,-0.223639,7
4,Upgrade AWS SDK v2 to version 2.17.77,-0.125230,-0.219510,0.187311,-1.210782,0.803214,-0.173714,0.523547,0.505062,0.190299,...,-0.313440,-0.118772,0.167825,1.265547,-0.016372,-0.814946,-0.053619,-0.114785,-0.680513,4


In [29]:
group_list = sorted(commit_message_vecs_hdf.kmean_label.unique())
for group_id in group_list:
    mask = commit_message_vecs_hdf.kmean_label == group_id
    print('group_id = %d' % group_id)
    min_size = min(sum(mask), 10)
    print( commit_message_vecs_hdf[mask].sample(min_size) )

group_id = 0
                                            message  message_C1  message_C2  \
900           CAMEL-13458: Fixed CS\r\nCloses #2883    0.012187   -0.226050   
2519                         CAMEL-10411 - Fixed CS    0.051756   -0.051947   
2296                                       Fixed CS    0.013442   -0.451354   
320                           Fixed CS and polished   -0.231155   -0.246721   
3081  Fixed CS. Fixes #708. Fixes #706. Fixes #705.   -0.330003   -0.282838   
1452                                       Fixed CS    0.013442   -0.451354   
2027                                       Fixed CS    0.013442   -0.451354   
1332                            CAMEL-12729: Fix CS   -0.036470    0.054958   
814                           CAMEL-13483: Fixed CS    0.051756   -0.051947   
3293                                    Fixed todos   -0.144285   -0.412879   

      message_C3  message_C4  message_C5  message_C6  message_C7  message_C8  \
900    -0.792995   -0.218833   -0.186

[10 rows x 32 columns]
group_id = 3
                                                message  message_C1  \
101                       Updating Bouncycastle to 1.65    0.333759   
2170  CAMEL-10650: moving global config to Camel con...    0.337740   
3998  CAMEL-7297 Add remote querying feature to came...    0.092317   
3994  Add unit test to support scenario concurrentCo...    0.162788   
301                          CAMEL-14354: Optimize core    0.339994   
875    CAMEL-13474: Move dataformats out of camel-core.    0.227779   
634   cleanups for camel-hdfs2 to camel-hdfs compone...    0.267349   
2672                      Upgraded roaster code changes   -0.217878   
926             CAMEL-13398 - Added docs for AWS-Lambda    0.125581   
6760  CAMEL-15186: Adding support for Workday Common...    0.189008   

      message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
101     0.159273   -0.223750   -0.137827    0.197837    0.129022    0.021468   
2170    0.008251   -0.

[10 rows x 32 columns]
group_id = 6
                                                message  message_C1  \
923   Add XML schema references for release 2.23.2\r...   -0.285010   
921   Update to Maven 3.6.1\r\n\r\nSigned-off-by: Gr...   -0.433818   
3958  CAMEL-7346: Remove redundant groupId definitio...   -0.524023   
1054  CAMEL-17104 - getObject operation support in p...   -0.078265   
1604  Update Maven wrapper to version 0.4.0\r\n\r\nS...   -0.393047   
3996  CAMEL-7285: Upgrade camel-jcr to Jackrabbit 2....   -0.152804   
2542  Update schema files with release 2.18.0 refere...   -0.331098   
1152  Add version 2.23.0 to XML schema files\r\n\r\n...   -0.206538   
4094  CAMEL-7067: Use message headers instead of exc...   -0.114908   
1574  Add XML schema references for release 2.19.5\r...   -0.285010   

      message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
923     1.058774   -0.022253   -0.072744   -0.198284   -0.045331    0.263867   
921     1.313900   -0.

[10 rows x 32 columns]
group_id = 9
                                                message  message_C1  \
263                                            Polished   -0.926854   
874                       camel3 - Remove unusued cruft   -0.264484   
699                            CAMEL-13628: Code review   -0.478597   
426   Merge pull request #3348 from shoukunhuai/patc...   -0.451661   
2048                                           Polished   -0.926854   
2473                                       First review   -0.300558   
3583                                           Polished   -0.926854   
3278                               CAMEL-7152: Polished   -0.399235   
3124                                           Polished   -0.926854   
4239                                          Polished.   -0.926854   

      message_C2  message_C3  message_C4  message_C5  message_C6  message_C7  \
263     0.114653    0.001802   -0.035378    0.035323   -0.254176   -0.231346   
874    -0.660675   -0.

In [30]:
print(len(okhttp_commits_df))
print(len(commit_message_vecs_hdf))

6798
6798


In [31]:
okhttp_commits_df['kmean_label'] = kmean.labels_
okhttp_commits_df.head()

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring,kmean_label
5,30494c20e104ffdd2d347966f836d079f5f97d36,aldettinger,aldettinger@gmail.com,2021-11-09 08:32:59,aldettinger,CAMEL-17178: Fix wrong jackson 2 HTTP links,f29fe7e9014f812b301780518ef7c503324d0619,2021-11-08 16:51:22,15.0,0,8
7,b73b55a039393621fb5dfa27ffbfcd011060d07c,Otavio Rodolfo Piske,opiske@redhat.com,2021-11-09 07:18:30,orpiske,CAMEL-17121: converted camel-soroush to use th...,464acde20df180c8bbd8b536afb90c9cfdc4dfcd,2021-11-08 14:42:42,16.0,0,3
9,868cf9016875075279da7c9616029467db80d019,Otavio Rodolfo Piske,opiske@redhat.com,2021-11-08 13:45:23,orpiske,CAMEL-17121: converted camel-smpp to the repea...,fb55460805d6049f4d3529ebf8a7b4a6002f14fa,2021-11-08 05:28:42,8.0,0,3
12,7933604702897ddf6c7664cfe16eec19983d48e7,Otavio Rodolfo Piske,opiske@redhat.com,2021-11-08 11:57:29,orpiske,CAMEL-17121: fixed time+iteration bounded supp...,dab936c75d24e565059f08898a0eb451a7f738d1,2021-11-08 04:54:39,7.0,0,7
22,c7c73b5730ca4c8e43c2a79860b83a339c79a035,Andrea Cosentino,ancosen@gmail.com,2021-11-09 05:37:37,oscerd,Upgrade AWS SDK v2 to version 2.17.77,2b42548de6a92181b603fde12001ac0c2684d482,2021-11-08 15:41:55,13.0,0,4


In [32]:
okhttp_commits_df['kmean_label'].value_counts()

1    1882
7    1662
3    1047
2     765
8     574
9     245
5     213
0     193
4     138
6      79
Name: kmean_label, dtype: int64

In [33]:
okhttp_commits_df['kmean_label'].isna().sum()

0

In [34]:
okhttp_commits_df[okhttp_commits_df['contains_refactoring'] == 1]

,sha,name,email,date,login,message,parent_sha,parent_date,time_taken,contains_refactoring,kmean_label
9015,4b2c7bf93c94461133dd4a58122a25a9b90f48a4,Claus Ibsen,claus.ibsen@gmail.com,2021-06-21 06:05:09,davsclaus,CAMEL-16734: Transacted in multicast EIPs shou...,aae087560693e9ace5ffb48a37b3c742b17fa60e,2021-06-20 11:56:57,18.0,1,7
10039,244d172d6c96314677ef9efec77efbeb6b5becc1,Hokutor,hwcamel@huawei.com,2021-06-18 04:22:49,hokutor,CAMEL-16717: Adding Huawei Cloud IAM Operation...,d4e56b46f59eb6589524a7aeefa207b9c1ce333d,2021-06-17 20:12:01,8.0,1,3
13367,fde51bfdce52572ee000e383293f2be42d62f22f,Paul Blanchaert,paul@search-solutions.net,2021-06-06 14:43:05,Paul-Blanchaert,CAMEL-16542 Camel-Solr: Refactoring the compon...,0a7e27640fd4c85aa8b8308a1cd86ce54c416256,2021-06-06 07:25:35,7.0,1,7
14524,3b3a9a00e401b97a17fcacf3d52c3af3969f2927,aldettinger,aldettinger@gmail.com,2021-05-26 08:46:37,aldettinger,CAMEL-16643: Dropped the useless dependency to...,06320a81abb35bfd69d5906eb1ec7d40cfed6572,2021-05-20 13:46:38,138.0,1,3
16624,1a2d9b78c042658a3053c2eb03632131f48fed5a,Claus Ibsen,claus.ibsen@gmail.com,2021-05-17 04:19:16,davsclaus,CAMEL-16618: camel-spring-boot - No CamelConte...,982ee8fdf3497c527ba6fec9d12ee28dc3cdbee3,2021-05-16 06:37:54,21.0,1,7
18795,322c021c0a02d942674f58bbccb3f22d001a2363,Dietrich Schulten,ds@escalon.de,2021-05-06 05:00:40,dschulten,CAMEL-16583: Reintroduce code samples in camel...,e818e0103490a106fa1538219f91a732ddebc562,2021-05-05 18:03:15,10.0,1,3
19847,39892fa352ab8bc4b39deae8dd2d6791c06c9881,Andrea Tarocchi,andrea.tarocchi@gmail.com,2021-05-02 14:17:18,valdar,CAMEL-16569: amel-kamelet don't check for dupl...,3fb0c758d5a65956795434a4ba8f21fa47652c66,2021-04-26 13:37:58,144.0,1,7
23114,5c31622dc19c2578e6268487e2f9b36706ac13e7,Otavio Rodolfo Piske,orpiske@users.noreply.github.com,2021-04-17 04:59:34,orpiske,(chores): removes OS-specific profiles for lev...,c91840a01352c663ff5338c45e096d256085331c,2021-04-16 16:11:11,12.0,1,7
23165,c089d566839f168a273847df3c2c98c9b8f7a987,Otavio Rodolfo Piske,orpiske@users.noreply.github.com,2021-04-16 04:20:38,orpiske,CAMEL-16400: split unit and integration tests ...,50009e87679348d4de02399327023baab585e424,2021-04-15 19:08:14,9.0,1,3
24250,8bfca9372d39fbf05172d5d0fdb35c484bb30c8f,Otavio Rodolfo Piske,orpiske@users.noreply.github.com,2021-04-12 04:27:02,orpiske,CAMEL-16400: split unit and integration tests ...,5074bcc02079a11746d5189a1c33a4295ecd3307,2021-04-11 17:12:27,11.0,1,3


In [44]:
grouped_main_commits = okhttp_commits_df.groupby(['kmean_label', 'login']).agg({'time_taken': ['mean', 'min', 'max']}).reset_index()
grouped_main_commits

kmean_label         login time_taken             
                                    mean   min    max
0             0  CandleCandle  97.000000  97.0   97.0
1             0   WillemJiang  38.214286   7.0  116.0
2             0   aldettinger  31.166667   7.0  117.0
3             0    anoordover  26.000000  26.0   26.0
4             0     davsclaus  32.190909   6.0  158.0
..          ...           ...        ...   ...    ...
616           9       tlehoux  10.000000  10.0   10.0
617           9        valdar   6.000000   6.0    6.0
618           9     vishal423  18.000000  18.0   18.0
619           9         woj-i  39.000000  39.0   39.0
620           9      zregvart  41.500000  16.0   67.0

[621 rows x 5 columns]

In [45]:
grouped_main_commits[grouped_main_commits['kmean_label']==3]

kmean_label           login time_taken             
                                      mean   min    max
69            3    CandleCandle      14.00  14.0   14.0
70            3       CodeSmell      71.00  71.0   71.0
71            3         DariusX      21.00   8.0   32.0
72            3    DenisIstomin       9.75   7.0   15.0
73            3           FloEc      13.00  13.0   13.0
..          ...             ...        ...   ...    ...
207           3  willianantunes      24.00  24.0   24.0
208           3         wingerx       8.00   8.0    8.0
209           3        xificurC       7.00   7.0    7.0
210           3          zhfeng      12.00   7.0   17.0
211           3        zregvart      59.00   7.0  165.0

[143 rows x 5 columns]

In [37]:
okhttp_commits_df_refactored = okhttp_commits_df[okhttp_commits_df['contains_refactoring'] == 1]
okhttp_commits_df_refactored.groupby(['kmean_label','login']).agg({'time_taken': ['mean', 'min', 'max']})

time_taken              
                                   mean    min    max
kmean_label login                                    
3           DenisIstomin            8.0    8.0    8.0
            aldettinger           138.0  138.0  138.0
            davsclaus               7.0    7.0    7.0
            dschulten              10.0   10.0   10.0
            hokutor                 8.0    8.0    8.0
            marcelloraffaele       11.0   11.0   11.0
            orpiske                11.0    9.0   13.0
7           Paul-Blanchaert         7.0    7.0    7.0
            davsclaus              15.5    8.0   21.0
            gnodet                 14.0   14.0   14.0
            lburgazzoli            21.0   19.0   23.0
            orpiske                12.0   12.0   12.0
            valdar                144.0  144.0  144.0
8           orpiske                 8.0    8.0    8.0